In [4]:
import psycopg2
import pandas as pd

from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import numpy as np
from sentence_transformers import SentenceTransformer


In [5]:
used_datasets = [
    'tweets',
    'spam',
    'plots',
    'amazon',
    'agnews',
    'physician_professor',
    'imdb',
    'fakenews',
    'yelp',
    'professor_teacher',
    'painter_architect',

]

conn = psycopg2.connect(
    dbname="label",
    user="postgres",
)

mapping_dicts = {}

In [6]:
for ud in used_datasets:
    print(ud)
    info_df = pd.read_sql(f'select count(*), class, label from {ud} group by class, label', conn)
    print(info_df)
    result = dict(zip(info_df['label'], info_df['class']))
    mapping_dicts[ud]= result
    print('\n')

tweets
   count  class     label
0   2363      0  positive
1   9178      1  negative


spam
   count  class label
0    747      1  spam
1   4825      0   ham


plots
   count  class    label
0    915      0   action
1   1030      1  romance


amazon
    count  class     label
0   99936      0  positive
1  100064      1  negative


agnews
   count  class       label
0  30000      0  Technology
1  30000      1    Business


physician_professor
   count  class      label
0  27238      1  physician
1  27238      0  professor


imdb
   count  class     label
0  25000      0  negative
1  25000      1  positive


fakenews
   count  class label
0  21417      0  true
1  23481      1  fake


yelp
   count  class     label
0  19000      0  negative
1  19000      1  positive


professor_teacher
   count  class      label
0  12294      1  professor
1  12294      0    teacher


painter_architect
   count  class      label
0   6118      0    painter
1   6118      1  architect




/var/folders/sl/1_4_3jg90p50ft92b5k6mgv00000gn/T/ipykernel_8967/1608051740.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  info_df = pd.read_sql(f'select count(*), class, label from {ud} group by class, label', conn)
/var/folders/sl/1_4_3jg90p50ft92b5k6mgv00000gn/T/ipykernel_8967/1608051740.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  info_df = pd.read_sql(f'select count(*), class, label from {ud} group by class, label', conn)
/var/folders/sl/1_4_3jg90p50ft92b5k6mgv00000gn/T/ipykernel_8967/1608051740.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not test

In [7]:

model = SentenceTransformer('all-MiniLM-L6-v2')

for k, v in mapping_dicts.items():
    query = f"SELECT * FROM {k}"  # Make sure `k` is sanitized to prevent SQL injection
    df = pd.read_sql(query, conn)

    embeddings = model.encode(df['content'].tolist(), show_progress_bar=True)
    
    silhouette_scores = []
    k_values = range(2, 10)  # Test clusters from 2 to 10
    
    for num_clusters in k_values:
        kmeans = KMeans(n_clusters=num_clusters, random_state=42)
        labels = kmeans.fit_predict(embeddings)
        score = silhouette_score(embeddings, labels)
        silhouette_scores.append(score)
    
    optimal_k = k_values[np.argmax(silhouette_scores)]
    print(f"Optimal number of clusters for table {k}: {optimal_k}")
    
    # Step 3: Perform clustering with the optimal number of clusters
    kmeans = KMeans(n_clusters=optimal_k, random_state=42)
    df['cluster'] = kmeans.fit_predict(embeddings)
    
    # Step 4: Save or process the updated DataFrame
    output_file = f"clustered_{k}.csv"
    df.to_csv(output_file, index=False)
    print(f"Clustered DataFrame for {k} saved to {output_file}")
    

/var/folders/sl/1_4_3jg90p50ft92b5k6mgv00000gn/T/ipykernel_8967/2515698421.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Batches:   0%|          | 0/361 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Optimal number of clusters for table tweets: 2
Clustered DataFrame for tweets saved to clustered_tweets.csv


/var/folders/sl/1_4_3jg90p50ft92b5k6mgv00000gn/T/ipykernel_8967/2515698421.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


...

Batches:   0%|          | 0/769 [00:00<?, ?it/s]

Optimal number of clusters for table professor_teacher: 2
Clustered DataFrame for professor_teacher saved to clustered_professor_teacher.csv


/var/folders/sl/1_4_3jg90p50ft92b5k6mgv00000gn/T/ipykernel_8967/2515698421.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Batches:   0%|          | 0/383 [00:00<?, ?it/s]

Optimal number of clusters for table painter_architect: 2
Clustered DataFrame for painter_architect saved to clustered_painter_architect.csv
